### Imports

In [1]:
import duckdb
from core.utils import duck_to_pandas, expand_column
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from google.cloud import storage
import re

ModuleNotFoundError: No module named 'core'

### Events

In [3]:
database_path = '/home/marcelo-borges/Documentos/Projetos/SIAN/laboratory/telemetry-validation/db/telemetria.duckdb'
query = "SELECT * FROM events"
df_events = duck_to_pandas(database_path, query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

### EDA

In [4]:
df_events.sample(10)

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,StartPosition,Value,EndDateTime,StartOdometerKilometres,StartDateTime,EventCategory,FuelUsedLitres,EndPosition,EndOdometerKilometres,LocationId
964698,1,41,-3393530750645328945,3320727677650052618,-106647426425259773,1514402039761068032,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.00000,2024-07-01T16:40:03Z,621501.50,2024-07-01T16:39:22Z,Detail,0.022500,NaN,NaN,NaN
223045,1,1,7651958308154428287,3294799041980858830,-106647426425259773,1498924246870650880,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.00000,2024-04-22T19:43:02Z,918953.10,2024-04-22T19:43:01Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",918953.10,NaN
363869,1,1,7651958308154428287,3300035707343279653,1491699302610141184,1498925011269795840,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.00000,2024-05-06T22:23:58Z,975730.00,2024-05-06T22:23:57Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",975730.00,NaN
495595,1,2,-4006354661462717098,3304346529747652974,-106647426425259773,1514472812174508032,"{'AgeOfReadingSeconds': 6, 'AltitudeMetres': N...",0.00000,2024-05-18T13:12:11Z,551665.30,2024-05-18T13:12:09Z,Detail,NaN,NaN,NaN,NaN
1147922,1,754,4745460876959782086,3326435778842124319,1491699300338892800,1498925011269795840,NaN,81.06801,2024-07-17T02:02:17Z,NaN,2024-07-17T01:49:43Z,Summary,NaN,NaN,NaN,NaN
720445,1,4,-3393530750645328945,3312091826316419596,-106647426425259773,1498924535635898368,NaN,0.00000,2024-06-08T10:07:55Z,NaN,2024-06-08T10:07:51Z,Detail,0.005344,NaN,NaN,NaN
981089,1,1,-4006354661462717098,3321164242569634778,-106647426425259773,1514402039761068032,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",0.00000,2024-07-02T20:53:29Z,621641.40,2024-07-02T20:53:28Z,Detail,NaN,NaN,NaN,NaN
242485,1,1,7651958308154428287,3295401121008470558,-106647426425259773,1498924535635898368,"{'AgeOfReadingSeconds': 3, 'AltitudeMetres': N...",0.00000,2024-04-24T10:39:25Z,12203.59,2024-04-24T10:39:24Z,Detail,NaN,"{'AgeOfReadingSeconds': 3, 'AltitudeMetres': N...",12203.60,NaN
1208967,1,1,1443761028319829598,3327018195170116827,-106647426425259773,1547008473392844800,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",1800.00000,2024-07-18T15:29:48Z,52354.09,2024-07-18T15:29:47Z,Detail,NaN,"{'AgeOfReadingSeconds': 0, 'AltitudeMetres': N...",52354.09,NaN
51829,1,1,7651958308154428287,3287417034853014560,-106647426425259773,1498924535635898368,"{'AgeOfReadingSeconds': 2, 'AltitudeMetres': N...",0.00000,2024-04-02T22:17:05Z,10042.91,2024-04-02T22:17:04Z,Detail,NaN,"{'AgeOfReadingSeconds': 2, 'AltitudeMetres': N...",10042.91,NaN


In [5]:
df_events.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,LocationId
count,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.238955e+06,1.205826e+06,9.181830e+05,381046.000000,6.378600e+05,1.206700e+04
mean,2.005445e+00,2.073852e+02,1.551935e+18,3.308199e+18,4.865914e+17,1.507137e+18,1.007198e+03,6.964791e+05,0.019021,6.924665e+05,1.508070e+18
std,1.317910e+01,1.397186e+03,5.584598e+18,1.285241e+16,1.217497e+18,1.386682e+16,4.812906e+03,5.094844e+05,0.045152,4.136210e+05,2.815040e+14
min,0.000000e+00,0.000000e+00,-8.078464e+18,3.275198e+18,-9.195140e+18,1.498924e+18,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,1.507979e+18
25%,1.000000e+00,1.000000e+00,-3.393531e+18,3.297818e+18,-1.066474e+17,1.498925e+18,0.000000e+00,5.489248e+05,0.003741,5.532453e+05,1.507980e+18
50%,1.000000e+00,3.000000e+00,1.443761e+18,3.308728e+18,-1.066474e+17,1.498925e+18,0.000000e+00,6.855360e+05,0.008728,8.499894e+05,1.507981e+18
75%,1.000000e+00,1.500000e+01,7.651958e+18,3.319500e+18,1.491699e+18,1.514417e+18,3.200000e-01,9.167366e+05,0.022444,9.222814e+05,1.507983e+18
max,8.950000e+02,9.269800e+04,8.678978e+18,3.327150e+18,9.206705e+18,1.547012e+18,3.111738e+04,4.050610e+06,5.566263,4.050610e+06,1.508963e+18


In [6]:
df_events.dtypes

TotalOccurances              int64
TotalTimeSeconds             int64
EventTypeId                  int64
EventId                      int64
DriverId                     int64
AssetId                      int64
StartPosition               object
Value                      float64
EndDateTime                 object
StartOdometerKilometres    float64
StartDateTime               object
EventCategory               object
FuelUsedLitres             float64
EndPosition                 object
EndOdometerKilometres      float64
LocationId                 float64
dtype: object

In [7]:
df_events.isna().sum()

TotalOccurances                  0
TotalTimeSeconds                 0
EventTypeId                      0
EventId                          0
DriverId                         0
AssetId                          0
StartPosition               320773
Value                        33129
EndDateTime                  26912
StartOdometerKilometres     320772
StartDateTime                    0
EventCategory                    0
FuelUsedLitres              857909
EndPosition                 601091
EndOdometerKilometres       601095
LocationId                 1226888
dtype: int64

### Uso de memória

#### original

In [8]:
specific_date = '2024-07-18'
# filtered_df = df_positions[df_positions['Timestamp'].dt.date == pd.to_datetime(specific_date).date()]
df_original = df_events[df_events['StartDateTime'].str.contains(specific_date)]

In [9]:
import builtins
memory_usage_bytes = df_original.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 58951920 bytes
Memória total usada pelo DataFrame: 56.22 MB
Memória total usada pelo DataFrame: 0.05 GB


In [10]:
df_original.to_csv('df_original.csv', sep=';',decimal=',')

#### sugerido

In [11]:
df_sugerido = df_original.copy()

In [12]:
start_position_expanded = expand_column(df_sugerido, 'StartPosition')
end_position_expanded = expand_column(df_sugerido, 'EndPosition')

In [13]:
df_expanded = pd.concat([df_sugerido.drop(columns=['StartPosition', 'EndPosition']), start_position_expanded.add_prefix('StartPosition_'), end_position_expanded.add_prefix('EndPosition_')], axis=1)


In [14]:
df_expanded.describe()

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,StartOdometerKilometres,FuelUsedLitres,EndOdometerKilometres,...,EndPosition_Hdop,EndPosition_Heading,EndPosition_Latitude,EndPosition_Longitude,EndPosition_NumberOfSatellites,EndPosition_OdometerKilometres,EndPosition_Pdop,EndPosition_PositionId,EndPosition_SpeedKilometresPerHour,EndPosition_SpeedLimit
count,60597.000000,60597.000000,6.059700e+04,6.059700e+04,6.059700e+04,6.059700e+04,60597.000000,48376.000000,11575.000000,37080.000000,...,37081.000000,37081.000000,37081.000000,37081.000000,37081.000000,37075.000000,37081.0,3.708100e+04,37081.000000,23495.000000
mean,1.596317,184.207023,7.567854e+17,3.327009e+18,2.443609e+17,1.537603e+18,1445.067649,666404.310789,0.013540,672653.788072,...,1.022842,172.926485,-16.737004,-49.288460,11.987837,672744.503670,0.0,3.327009e+18,22.309876,53.759098
std,9.689379,990.335863,3.880342e+18,9.175486e+13,3.056904e+18,1.806712e+16,3746.143316,234014.461953,0.023141,240205.128034,...,0.737273,103.877560,0.050117,0.044617,0.212571,240094.263288,0.0,9.001194e+13,12.358762,10.899364
min,0.000000,0.000000,-8.078464e+18,3.326779e+18,-8.976664e+18,1.498924e+18,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,-16.976239,-49.454449,5.000000,0.053000,0.0,3.326779e+18,0.000000,30.000000
25%,1.000000,1.000000,-3.393531e+18,3.326938e+18,-1.066474e+17,1.547008e+18,0.000000,580294.675000,0.003500,580331.100000,...,1.000000,79.000000,-16.766032,-49.318716,12.000000,580331.600000,0.0,3.326939e+18,13.663000,40.000000
50%,1.000000,2.000000,1.443761e+18,3.327016e+18,-1.066474e+17,1.547009e+18,800.000000,708414.900000,0.006500,733710.900000,...,1.000000,179.000000,-16.728280,-49.283877,12.000000,733711.100000,0.0,3.327015e+18,23.254000,60.000000
75%,1.000000,13.000000,1.443761e+18,3.327092e+18,3.118870e+17,1.547011e+18,1800.000000,835459.500000,0.016000,835553.425000,...,1.000000,253.000000,-16.702322,-49.258800,12.000000,835553.438000,0.0,3.327090e+18,31.514000,60.000000
max,527.000000,33309.000000,8.678978e+18,3.327150e+18,7.405822e+18,1.547012e+18,29313.470588,982848.200000,0.579000,982848.300000,...,25.000000,359.000000,-16.650273,-49.127992,12.000000,982848.300000,0.0,3.327150e+18,60.727000,110.000000


In [15]:
null_count = df_expanded.isnull().sum()
total_registers = df_expanded.shape[0]
percent_null = (null_count / total_registers) * 100

In [16]:
formatted_percent_null = [f"{percent:.2f}%" for percent in percent_null]

In [17]:
percent_null_df = pd.DataFrame({
    'Coluna': df_expanded.columns,
    'Percentual de Nulos': formatted_percent_null
})

In [18]:
percent_null_df.sort_values('Percentual de Nulos', ascending=False)

,Coluna,Percentual de Nulos
13,LocationId,99.38%
33,EndPosition_AltitudeMetres,99.38%
15,StartPosition_AltitudeMetres,99.38%
11,FuelUsedLitres,80.90%
48,EndPosition_SpeedLimit,61.23%
30,StartPosition_SpeedLimit,51.57%
43,EndPosition_OdometerKilometres,38.82%
34,EndPosition_AssetId,38.81%
35,EndPosition_DriverId,38.81%
36,EndPosition_FormattedAddress,38.81%


#### Sugestão de retirada de coluna

In [19]:
df_expanded.drop(columns=['EndPosition_AltitudeMetres','StartPosition_AltitudeMetres'], inplace=True)

In [20]:
df_expanded['EndDateTime'] = pd.to_datetime(df_expanded['EndDateTime'], errors='coerce')
df_expanded['StartDateTime'] = pd.to_datetime(df_expanded['StartDateTime'], errors='coerce')


In [21]:
float_to_int_columns = [
    'StartPosition_AgeOfReadingSeconds',
    'StartPosition_AssetId',
    'StartPosition_DriverId',
    'StartPosition_NumberOfSatellites',
    'StartPosition_PositionId',
    'EndPosition_AgeOfReadingSeconds',
    'EndPosition_AssetId',
    'EndPosition_DriverId',
    'EndPosition_NumberOfSatellites',
    'EndPosition_PositionId'
]

In [22]:
for column in float_to_int_columns:
    df_expanded[column] = df_expanded[column].astype('Int64', errors='ignore')


In [23]:
data_types_after_casting = df_expanded.dtypes
data_types_after_casting

TotalOccurances                                       int64
TotalTimeSeconds                                      int64
EventTypeId                                           int64
EventId                                               int64
DriverId                                              int64
AssetId                                               int64
Value                                               float64
EndDateTime                             datetime64[ns, UTC]
StartOdometerKilometres                             float64
StartDateTime                           datetime64[ns, UTC]
EventCategory                                        object
FuelUsedLitres                                      float64
EndOdometerKilometres                               float64
LocationId                                          float64
StartPosition_AgeOfReadingSeconds                     Int64
StartPosition_AssetId                                 Int64
StartPosition_DriverId                  

In [24]:
import builtins
memory_usage_bytes = df_expanded.memory_usage(deep=True)
memory_values = list(memory_usage_bytes)
total_memory_usage_bytes = builtins.sum(memory_values)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_bytes} bytes")
total_memory_usage_mb = total_memory_usage_bytes / (1024 ** 2)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_mb:.2f} MB")
total_memory_usage_gb = total_memory_usage_bytes / (1024 ** 3)
print(f"Memória total usada pelo DataFrame: {total_memory_usage_gb:.2f} GB")

Memória total usada pelo DataFrame: 55183892 bytes
Memória total usada pelo DataFrame: 52.63 MB
Memória total usada pelo DataFrame: 0.05 GB


In [25]:
df_expanded.to_csv('df_expanded.csv', sep=';',decimal=',')

In [28]:
df_expanded

,TotalOccurances,TotalTimeSeconds,EventTypeId,EventId,DriverId,AssetId,Value,EndDateTime,StartOdometerKilometres,StartDateTime,...,EndPosition_Latitude,EndPosition_Longitude,EndPosition_NumberOfSatellites,EndPosition_OdometerKilometres,EndPosition_Pdop,EndPosition_PositionId,EndPosition_Source,EndPosition_SpeedKilometresPerHour,EndPosition_SpeedLimit,EndPosition_Timestamp
1178358,1,1,1443761028319829598,3326778596109246960,-106647426425259773,1514480648201252864,1700.000000,2024-07-18 00:00:02+00:00,496665.80,2024-07-18 00:00:01+00:00,...,-16.685209,-49.296366,12,496665.7500,0.0,3326778604278690304,Gps,22.605,NaN,2024-07-18T00:00:03Z
1178359,1,2,1443761028319829598,3326778594953330951,-106647426425259773,1547008786057342976,2000.000000,2024-07-18 00:00:03+00:00,646893.00,2024-07-18 00:00:01+00:00,...,-16.746432,-49.281740,12,646893.0630,0.0,3326778606246849536,Gps,28.636,60.0,2024-07-18T00:00:04Z
1178360,1,3,1443761028319829598,3326778598789211921,-106647426425259773,1547008473392844800,1800.000000,2024-07-18 00:00:05+00:00,52205.37,2024-07-18 00:00:02+00:00,...,-16.710966,-49.311278,12,52205.4141,0.0,3326778615969628672,Gps,37.613,60.0,2024-07-18T00:00:06Z
1178361,118,526,-3263360059171983396,3326778602345042490,-106647426425259773,1547008786057342976,100.000000,2024-07-18 01:20:56+00:00,NaN,2024-07-18 00:00:03+00:00,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1178362,252,1244,7651958308154428287,3326778616345718186,-106647426425259773,1547008786057342976,0.000000,2024-07-18 01:21:47+00:00,NaN,2024-07-18 00:00:06+00:00,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238950,1,1,-3393530750645328945,3327149605434196582,-106647426425259773,1547008220472225792,0.000000,2024-07-18 23:59:45+00:00,541658.60,2024-07-18 23:59:44+00:00,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1238951,1,27817,6005371287290190191,3327149606143658666,-106647426425259773,1498924246870650880,28261.192157,2024-07-19 07:43:21+00:00,928578.90,2024-07-18 23:59:44+00:00,...,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
1238952,1,3,1443761028319829598,3327149630824879864,-106647426425259773,1547008786057342976,1700.000000,2024-07-18 23:59:53+00:00,647106.60,2024-07-18 23:59:50+00:00,...,-16.698340,-49.252566,12,647106.6000,0.0,3327149651022156288,Gps,53.732,60.0,2024-07-18T23:59:54Z
1238953,1,5,7651958308154428287,3327149636933758522,1491699300338892800,1498925011269795840,0.000000,2024-07-18 23:59:56+00:00,982848.20,2024-07-18 23:59:51+00:00,...,-16.671192,-49.284878,12,982848.3000,0.0,3327149664652538880,Gps,36.436,40.0,2024-07-18T23:59:57Z


In [27]:
df_original.to_csv('events_original.csv', sep=';',decimal=',')
df_expanded.to_csv('events_expanded.csv', sep=';',decimal=',')